In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from verbio import readers, preprocessing, temporal

from scipy import stats
import scipy
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import os
from scipy.io import wavfile 
from scipy.ndimage import median_filter

from random import shuffle
import neurokit2 as nk

In [5]:
pts = [i for i in range(1, 74, 1)]

win_size = 15.0
stride = 1.0

In [27]:
percentages = []
for pt in pts:

    base_path = f'/home/jason/hubbs/project_verbio/data/raw_data/P{pt:03d}/'

    for session in ['PRE', 'POST']:

        session_path = os.path.join(base_path, session)
        eda_path = os.path.join(session_path, 'E4_EDA_PPT.xlsx')
        hr_path = os.path.join(session_path, 'E4_HR_PPT.xlsx')
        annotation_path = os.path.join(session_path, 'MANUAL_ANNOTATION_PPT.xlsx')
        try:
            annotation_df = pd.read_excel(annotation_path, engine='openpyxl')

            annotations_r1 = annotation_df['R1'].to_numpy()
            annotations_r2 = annotation_df['R2'].to_numpy()
            
            annotation_times = annotation_df['Time (s)'].to_numpy()
        except IOError:
            continue

        try:
            eda_df = pd.read_excel(eda_path, engine='openpyxl')         

            eda_times = eda_df['Time (s)'].to_numpy()
            original_eda = eda_df['EDA'].to_numpy()
            eda_signal = eda_df['EDA'].to_numpy()
            
            # Filter EDA with NK
            sr = 4
            order = 4
            w0 = 1.5 # Cutoff frequency 
            w0 = 2 * np.array(w0) / sr 

            eda_signal = nk.signal_sanitize(eda_signal)
            b, a = scipy.signal.butter(N=order, Wn=w0, btype='lowpass', analog=False, output='ba')
            eda_signal = scipy.signal.filtfilt(b, a, eda_signal)
            eda_signal = nk.signal_smooth(eda_signal, method='convolution', kernel='blackman', size=16)
            
            eda_decomp = nk.eda_phasic(eda_signal, sampling_rate=sr)
            
            eda_peaks, info = nk.eda_peaks(
                eda_decomp['EDA_Phasic'].values,
                sampling_rate=sr,
                method='biosppy',
                amplitude_min=0.1
            )
            
            peak_indices = info['SCR_Peaks'] 
            eda_tonic = eda_decomp['EDA_Tonic']
            
        except IOError:
            continue
            
        try:
            hr_df = pd.read_excel(hr_path, engine='openpyxl')
            
            hr_times = hr_df['Time (s)'].to_numpy()
            hr_data = hr_df['HR'].to_numpy()
            
            hr_data_grad = np.gradient(hr_data)
        
            grad_peaks, _ = scipy.signal.find_peaks(hr_data_grad, height=0.3)
            
            
            
        except:
            continue
            
        cluster_len = 20.0
        cluster_stride = 5.0

#         fig = plt.figure(figsize=(20,6))
    
#         ax1 = fig.add_subplot(211)
#         ax1.plot(eda_times, eda_signal)
#         ax1.grid()
#         ax1.set_ylabel('EDA SCRs')
        
#         for index in peak_indices:
#             plt.axvline(x=eda_times[index], color='red')
            
#         ax2 = fig.add_subplot(212)
#         ax2.plot(hr_times, hr_data_grad)
#         ax2.grid()
#         ax2.set_ylabel('HR Grad')
        
#         for index in grad_peaks:
#             plt.axvline(x=hr_times[index], color='red')
#         plt.axhline(y=0, color='black')
            
#         plt.show()
#         plt.clf()
        
        slices = temporal.time_slices(eda_times, cluster_len, cluster_stride)
        
        counts = []
        timestamps = []
        t = cluster_len
        hop = cluster_stride
        for t0, tk in slices:
            window_counts = ((t0 < peak_indices) & (peak_indices < tk)).sum()
            counts.append(window_counts)
            timestamps.append(t)
            t += hop
            
        slices = temporal.time_slices(hr_times, cluster_len, cluster_stride)
            
        hr_counts = []
        hr_timestamps = []
        t = cluster_len
        hop = cluster_stride
        for t0, tk in slices:
            window_counts = ((t0 < grad_peaks) & (grad_peaks < tk)).sum()
            hr_counts.append(window_counts)
            hr_timestamps.append(t)
            t += hop
            

        counts = np.array(counts)
        counts = preprocessing.binarize(counts, 4)
            
        hr_counts = np.array(hr_counts)
        hr_counts = preprocessing.binarize(hr_counts, 1)
        
        percentages.append(len(np.where(counts*hr_counts == 1)[0])/counts.shape[0])

            
#         fig = plt.figure(figsize=(20,6))
#         ax1 = fig.add_subplot(211)
#         ax1.step(timestamps, counts)
#         ax1.grid()
#         ax1.set_ylabel('EDA Freqs')
#         ax2 = fig.add_subplot(212)
#         ax2.step(hr_timestamps, hr_counts)
#         ax2.grid()
#         ax2.set_ylabel('HR Grad Freqs')
#         ax2.set_xlabel('Time (s)')
#         plt.show()

        

In [28]:
print(percentages)
print(sum(percentages)/len(percentages))

[0.0, 0.0, 0.0, 0.0, 0.6326530612244898, 0.0, 0.05555555555555555, 0.2608695652173913, 0.08, 0.0851063829787234, 0.13513513513513514, 0.1875, 0.05454545454545454, 0.0, 0.09090909090909091, 0.21428571428571427, 0.0, 0.2553191489361702, 0.0, 0.23636363636363636, 0.01818181818181818, 0.0196078431372549, 0.09090909090909091, 0.3508771929824561, 0.017857142857142856, 0.25, 0.0, 0.0, 0.4090909090909091, 0.0, 0.0, 0.12, 0.24528301886792453, 0.26666666666666666, 0.19642857142857142, 0.0, 0.22807017543859648, 0.019230769230769232, 0.09302325581395349, 0.1346153846153846, 0.21818181818181817, 0.07692307692307693, 0.0, 0.05357142857142857, 0.24390243902439024, 0.0, 0.17647058823529413, 0.029411764705882353, 0.38461538461538464, 0.0, 0.2222222222222222, 0.10714285714285714, 0.0625, 0.03571428571428571, 0.08928571428571429, 0.38181818181818183, 0.0, 0.0, 0.0, 0.03571428571428571, 0.0, 0.0, 0.1111111111111111, 0.05357142857142857, 0.03125, 0.35714285714285715, 0.0, 0.021739130434782608, 0.1627906976